In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# 開始年
start_y=2014
# dfにするファイル名
files=[
'tp20151211-01_1.xls',
'tp20151211-01_3.xls',
'tp20151211-01_1.xls',
'tp20151211-01_3.xls',
'tp20180314-01_1.xls',
'tp20171208-01_3.xls',
'tp20180314-01_1.xls',
'tp20171208-01_3.xls',
'tp20190904-01_01.xls',
'tp20190904-01_03.xls',
'tp20200122-01_01.xls',
'tp20200122-01_03.xls',
'tp20210218-01_01.xlsx',
'tp20210218-01_03.xlsx',
'tp20220201-01_01.xlsx',
'tp20211210-01_03.xlsx']
for i,name in enumerate(files):
    print(start_y+i//2,name)

2014 tp20151211-01_1.xls
2014 tp20151211-01_3.xls
2015 tp20151211-01_1.xls
2015 tp20151211-01_3.xls
2016 tp20180314-01_1.xls
2016 tp20171208-01_3.xls
2017 tp20180314-01_1.xls
2017 tp20171208-01_3.xls
2018 tp20190904-01_01.xls
2018 tp20190904-01_03.xls
2019 tp20200122-01_01.xls
2019 tp20200122-01_03.xls
2020 tp20210218-01_01.xlsx
2020 tp20210218-01_03.xlsx
2021 tp20220201-01_01.xlsx
2021 tp20211210-01_03.xlsx


なぜ複雑な処理をするかというと、年度ごとに生産企業名が変わっていたり、年度途中で追加、廃止されたりしていて、ndbのほうとラグが出ているために、うまくjoinできない場合があるから

header

In [3]:
# header for -2014
str="区分	薬価基準収載医薬品コード	成分名	規格	品名			メーカー名	診療報酬において加算等の算定対象となる後発医薬品	先発医薬品	同一剤形・規格の後発医薬品がある先発医薬品	薬価	経過措置による使用期限	備考"
str=str.split()
print(str)
header0=['区分', '薬価基準収載医薬品コード', '成分名', '規格',"局方","麻薬", '品名', 'メーカー名', '診療報酬において加算等の算定対象となる後発医薬品', '先発医薬品', '同一剤形・規格の後発医薬品がある先発医薬品', '薬価', '経過措置による使用期限', '備考']

['区分', '薬価基準収載医薬品コード', '成分名', '規格', '品名', 'メーカー名', '診療報酬において加算等の算定対象となる後発医薬品', '先発医薬品', '同一剤形・規格の後発医薬品がある先発医薬品', '薬価', '経過措置による使用期限', '備考']


In [4]:
# 2015 1
header0=['区分', '薬価基準収載医薬品コード', '成分名', '規格',"局方","麻薬","kome",'品名', 'メーカー名', '診療報酬において加算等の算定対象となる後発医薬品', '先発医薬品', '同一剤形・規格の後発医薬品がある先発医薬品', '薬価', '経過措置による使用期限', '備考',"a","b","c","d"]
# 2015 3
header1=['区分', '薬価基準収載医薬品コード', '成分名', '規格',"局方","麻薬","kome",'品名', 'メーカー名', '診療報酬において加算等の算定対象となる後発医薬品', '先発医薬品', '同一剤形・規格の後発医薬品がある先発医薬品', '薬価', '経過措置による使用期限', '備考']
# -2018
header2=['区分', '薬価基準収載医薬品コード', '成分名', '規格',"局方","麻薬","kome", '品名', 'メーカー名', '診療報酬において加算等の算定対象となる後発医薬品', '先発医薬品', '同一剤形・規格の後発医薬品がある先発医薬品', '薬価', '経過措置による使用期限', '備考']

In [5]:
header0=['区分', '薬価基準収載医薬品コード', '成分名', '規格',"局方","麻薬","kome",'品名', 'メーカー名', '診療報酬において加算等の算定対象となる後発医薬品', '先発医薬品', '同一剤形・規格の後発医薬品がある先発医薬品', '薬価', '経過措置による使用期限', '備考',"a","b","c","d"]
header1=['区分', '薬価基準収載医薬品コード', '成分名', '規格',"局方","麻薬","kome",'品名', 'メーカー名', '診療報酬において加算等の算定対象となる後発医薬品', '先発医薬品', '同一剤形・規格の後発医薬品がある先発医薬品', '薬価', '経過措置による使用期限', '備考']
data_i=pd.read_excel(f"./datafiles/tp20151211-01_1.xls",names=header0)
data_i=pd.read_excel(f"./datafiles/tp20151211-01_3.xls",names=header1)
data_i


,区分,薬価基準収載医薬品コード,成分名,規格,局方,麻薬,kome,品名,メーカー名,診療報酬において加算等の算定対象となる後発医薬品,先発医薬品,同一剤形・規格の後発医薬品がある先発医薬品,薬価,経過措置による使用期限,備考
0,外用薬,1112700X1011,ハロタン,１ｍＬ,局,NaN,NaN,ハロタン,NaN,NaN,NaN,NaN,54.4,NaN,NaN
1,外用薬,1114700X1016,麻酔用エーテル,１ｍＬ,局,NaN,NaN,麻酔用エーテル,NaN,NaN,NaN,NaN,9.4,NaN,NaN
2,外用薬,1116700X1010,亜酸化窒素,１ｇ,局,NaN,NaN,亜酸化窒素,NaN,NaN,NaN,NaN,3.9,NaN,NaN
3,外用薬,1116700X1045,亜酸化窒素,１ｇ,局,NaN,NaN,マルワ亜酸化窒素,和歌山酸素,NaN,NaN,NaN,4.3,NaN,NaN
4,外用薬,1116700X1053,亜酸化窒素,１ｇ,局,NaN,※,液化亜酸化窒素（日本エア・リキード）,日本エア・リキード,NaN,NaN,NaN,4.3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,外用薬,8219701S1025,フェンタニルクエン酸塩,１ｍｇ１枚,NaN,麻,NaN,フェントステープ１ｍｇ,久光製薬,NaN,先発品,NaN,586.9,NaN,NaN
2610,外用薬,8219701S2021,フェンタニルクエン酸塩,２ｍｇ１枚,NaN,麻,NaN,フェントステープ２ｍｇ,久光製薬,NaN,先発品,NaN,1094.0,NaN,NaN
2611,外用薬,8219701S3028,フェンタニルクエン酸塩,４ｍｇ１枚,NaN,麻,NaN,フェントステープ４ｍｇ,久光製薬,NaN,先発品,NaN,2039.0,NaN,NaN
2612,外用薬,8219701S4024,フェンタニルクエン酸塩,６ｍｇ１枚,NaN,麻,NaN,フェントステープ６ｍｇ,久光製薬,NaN,先発品,NaN,2935.1,NaN,NaN


In [6]:
firm_data=pd.DataFrame()
for i,name in enumerate(files):
    print(start_y+i//2,name)
    this_year = start_y+i//2
    last_year = (start_y+i//2)-1
    if this_year<=2015 and i%2==0:
        data_i=pd.read_excel(f"./datafiles/{name}",names=header0)
    elif this_year<=2015 and i%2==1:
        data_i=pd.read_excel(f"./datafiles/{name}",names=header1)
    else:
        data_i=pd.read_excel(f"./datafiles/{name}",names=header2)
    data_i['year']=start_y+i//2
    firm_data = pd.concat([firm_data,data_i],axis=0,ignore_index=True)
    # continue
    # 1年に1回のみ実行
    if i%2==1 and this_year>=2015:
        # 現在のDataFrameから年とIDの組み合わせを取得
        # firm_data_2y=firm_data[firm_data['year']>=(last_year)]
        firm_data_last=firm_data[firm_data['year']==(last_year)]
        firm_data_this=firm_data[firm_data['year']==(this_year)]
        set_this=set(firm_data_this["薬価基準収載医薬品コード"])
        set_last=set(firm_data_last["薬価基準収載医薬品コード"])
        # existing_combinations = set(zip(firm_data_2y['year'], firm_data_2y["薬価基準収載医薬品コード"]))
        # this yearにだけある
        set_this_m_last=set_this-set_last
        # last year にだけある
        set_last_m_this=set_last-set_this

        # 去年に、今年のIDがない場合、今年のIDを持つ行をコピー、昨年に対応する行を作成
        new_rows = []
        for id in set_this_m_last:
            this_year_row = firm_data[(firm_data['year'] == this_year) & (firm_data["薬価基準収載医薬品コード"] == id)]
            new_row = this_year_row.copy()
            new_row['year'] = last_year
            new_rows.append(new_row)
        if this_year in [2018,2019,2020]:
            # 今年に、last yearのIDがない場合、last yearのIDを持つ行をコピーしthis yearに対応する行を作成
            for id in set_last_m_this:
                    last_year_row = firm_data[(firm_data['year'] == last_year) & (firm_data["薬価基準収載医薬品コード"] == id)]
                    new_row = last_year_row.copy()
                    new_row['year'] = this_year
                    new_rows.append(new_row)
        # 新しい行を現在のDataFrameに追加
        firm_data = pd.concat([firm_data] + new_rows,axis=0, ignore_index=True)
    

2014 tp20151211-01_1.xls
2014 tp20151211-01_3.xls
2015 tp20151211-01_1.xls
2015 tp20151211-01_3.xls
2016 tp20180314-01_1.xls
2016 tp20171208-01_3.xls
2017 tp20180314-01_1.xls
2017 tp20171208-01_3.xls
2018 tp20190904-01_01.xls
2018 tp20190904-01_03.xls
2019 tp20200122-01_01.xls
2019 tp20200122-01_03.xls
2020 tp20210218-01_01.xlsx
2020 tp20210218-01_03.xlsx
2021 tp20220201-01_01.xlsx
2021 tp20211210-01_03.xlsx


In [7]:
# sorted(list(set(firm_data["メーカー名"])))

In [8]:
import re
import unicodedata
# make name clean
maker_set=sorted(list(set(firm_data["メーカー名"].fillna("統一"))))
maker2cleaned = {}
cleaned=[]
# テキストごとに処理を行うループ
for maker in maker_set:
    # クリーン処理を実行
    cleaned_maker = unicodedata.normalize("NFKC", maker)
    # \u3000を空白に置換
    cleaned_maker = re.sub("\u3000", " ", cleaned_maker)
    #　空白を削除
    cleaned_maker = re.sub(" ", "", cleaned_maker)
    # ,を削除
    cleaned_maker = re.sub(",", "", cleaned_maker)
    cleaned.append(cleaned_maker)
    # クリーン前のテキストをキー、クリーン後のテキストを値として辞書に追加
    maker2cleaned[maker] = cleaned_maker

maker_name=dict([# ('大塚製薬', '大塚製薬工場'),
# ('持田製薬', '持田製薬販売'),
# ('ブリストル・マイヤーズ', 'ブリストル・マイヤーズスクイブ'),
# ('サンド', 'サンドファーマ'),
# ('日医工', '日医工岐阜工場'),
('富士製薬', '富士製薬工業'),
# ('グラクソ・スミスクライン', 'グラクソ・スミスクライン・コンシューマー・ヘルスケア・ジャパン')
# ('ニプロ', 'ニプロパッチ'),
('エルメッドエーザイ','エルメッド'),
# ('ニプロ', 'ニプロESファーマ'),
('長生堂', '長生堂製薬'),
('東光薬品', '東光薬品工業'),
('岩城', '岩城製薬'),
('共和薬品', '共和薬品工業'),
# ('寿製薬', '千寿製薬'),
# ('日医工', '日医工サノフィ'),
# ('ファイザー', 'ファイザーUPJ'),
('高田', '高田製薬'),
# ('日医工', '日医工岐阜'),
# ('サノフィ', '日医工サノフィ'),
# ('第一三共', '第一三共エスファ'),
('大原', '大原薬品工業'),
('日医工岐阜', '日医工岐阜工場'),
# ('エーザイ', 'エルメッドエーザイ'),
# ('日医工', '日医工ファーマ'),
('沢井', '沢井製薬'),
('鶴原', '鶴原製薬'),
# ('ニプロ', 'ニプロファーマ'),
("協和発酵キリン","協和キリン")
])
firm_data["メーカー名"]=firm_data["メーカー名"].map(maker2cleaned).replace(maker_name)

In [9]:
firm_data.drop("薬価",inplace=True,axis=1)
# firm_data_o=firm_data.copy()
firm_data["メーカー名"]=firm_data["メーカー名"]
unified_data=firm_data[firm_data["メーカー名"].isna()]
firm_data_w_maker=firm_data[firm_data["メーカー名"].notna()]
firm_data_w_maker.shape

(105641, 19)

In [10]:
firm_data.loc[firm_data["メーカー名"].isna(),"メーカー名"]="統一"

In [11]:
unified_data.shape

(9283, 19)

In [12]:
# unified_data.to_csv("tmp.csv",encoding="utf_8_sig")

In [13]:
print("initial",len(set(firm_data["薬価基準収載医薬品コード"])),len(set(firm_data["品名"])))

initial 17912 17929


大正薬品工業は名前を変えた

In [14]:
firm_data_w_maker["year"]

1         2014
2         2014
3         2014
4         2014
5         2014
          ... 
114919    2020
114920    2020
114921    2020
114922    2020
114923    2020
Name: year, Length: 105641, dtype: int64

In [15]:
data_ndb=pd.read_csv("../merged/len3_ndb_blp_DN.csv",encoding="shift-jis",index_col=0)
# data_n=data_n[data_n["year"]>=2018]
data_ndb["医薬品コード"]=data_ndb["医薬品コード"].astype(int)
print("initial",len(set(data_ndb["薬価基準収載医薬品コード"])),len(set(data_ndb["医薬品コード"])),len(set(data_ndb["医薬品名"])))

initial 2671 2743 2744


In [16]:
data_ndb.columns

Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', '後発品区分', '総計',
       'year', 'in_hospital',
       ...
       'id_625.0', 'id_629.0', 'id_799.0', 'q_share', 'r_share',
       'generic_share_q', 'generic_share_r', 'lag_generic_share_q',
       'mean_price_g', 'mean_price_b'],
      dtype='object', length=112)

In [17]:
len(set(data_ndb["薬価基準収載医薬品コード"]) - set(firm_data["薬価基準収載医薬品コード"]))

0

In [18]:
data_ndb.duplicated(subset=['year', "医薬品コード","in_hospital"],keep=False).sum()

0

year を入れてもうまくいく

In [19]:
dups=firm_data[firm_data.duplicated(subset=["year","品名","薬価基準収載医薬品コード","メーカー名"],keep=False)]
# print(dups[["year","薬価基準収載医薬品コード","品名","メーカー名"]])
print(dups["year"].value_counts())

Series([], Name: year, dtype: int64)


In [20]:
dups=data_ndb[data_ndb.duplicated(subset=["year","医薬品コード","in_hospital"],keep=False)]
print(dups[["year","薬価基準収載医薬品コード","医薬品名","医薬品コード","in_hospital","oral"]])
data_ndb.shape

Empty DataFrame
Columns: [year, 薬価基準収載医薬品コード, 医薬品名, 医薬品コード, in_hospital, oral]
Index: []


(25798, 112)

In [21]:
data_merged=data_ndb.merge(firm_data,on=["薬価基準収載医薬品コード","year"],how="left",indicator=True)
# data_merged.loc[data_merged['_merge'] == 'left_only', 'メーカー名'] = data_merged.loc[data_merged['_merge'] == 'left_only','薬価基準収載医薬品コード'].map(code2maker)
# data_left_only=data_ndb.merge(firm_data,on=["薬価基準収載医薬品コード","year"],how="left",indicator=True).query('_merge == "left_only"')
print(data_merged.shape,data_merged.shape)
print(data_merged["_merge"].value_counts())

(25798, 130) (25798, 130)
both          25798
left_only         0
right_only        0
Name: _merge, dtype: int64


In [22]:
dups=data_merged[data_merged.duplicated(subset=["year","医薬品コード","in_hospital"],keep=False)]
print(dups[["year","薬価基準収載医薬品コード","医薬品名","医薬品コード","メーカー名","in_hospital","oral"]])

Empty DataFrame
Columns: [year, 薬価基準収載医薬品コード, 医薬品名, 医薬品コード, メーカー名, in_hospital, oral]
Index: []


In [23]:
len(set(data_merged.loc[data_merged["メーカー名"].isna(),"薬価基準収載医薬品コード"])),len(set(data_merged.loc[data_merged["メーカー名"].isna(),"医薬品コード"])),len(set(data_merged.loc[data_merged["メーカー名"].isna(),"医薬品名"]))

(0, 0, 0)

In [24]:
data_merged.loc[data_merged["メーカー名"].isna(),"医薬品名"].value_counts()

Series([], Name: 医薬品名, dtype: int64)

In [25]:
for id in set(data_merged.loc[data_merged["メーカー名"].isna(),"薬価基準収載医薬品コード"]):
    maker=set(firm_data.loc[firm_data["薬価基準収載医薬品コード"]==id,"メーカー名"])
    year=set(firm_data.loc[firm_data["薬価基準収載医薬品コード"]==id,"year"])
    year_merged=set(data_merged.loc[data_merged["薬価基準収載医薬品コード"]==id,"year"])
    if len(maker)>1:
        print(id,year,"ym",year_merged)
        print(maker)

In [26]:
len(set(data_merged.loc[data_merged["メーカー名"].isna(),"薬価基準収載医薬品コード"])),len(set(data_merged.loc[data_merged["メーカー名"].isna(),"医薬品コード"])),len(set(data_merged.loc[data_merged["メーカー名"].isna(),"医薬品名"]))

(0, 0, 0)

In [27]:
# calculate the mean of oral
data_merged["oral_per"] = data_merged.groupby(["year","薬効分類"])["oral"].transform("mean")
data_merged.loc[(data_merged["薬効分類"]==114)& (data_merged["year"]==2017),"oral_per"]

8376    0.665198
8377    0.665198
8378    0.665198
8379    0.665198
8380    0.665198
          ...   
8598    0.665198
8599    0.665198
8600    0.665198
8601    0.665198
8602    0.665198
Name: oral_per, Length: 227, dtype: float64

In [28]:
# add self and nonself


# もとのデータに結合
data_merged=data_merged.rename(columns={"後発品区分":"generic"})
data_merged["brand"]=1-data_merged["generic"]
data_merged.loc[:,"Pharmacopoeia"]=data_merged["局方"].replace("局",1).fillna(0)
data_merged["otc"]=0
data4add=data_merged.copy()
data4add.drop_duplicates(subset=["year","薬効分類"],keep="first",inplace=True)
print(data4add.shape)
add_data=pd.DataFrame()
for index,row in data4add.iterrows():
    nonself_row=row.copy()
    nonself_row.update({"医薬品名": "nonself", "メーカー名": "nonself","総計":row["nonself_quantity"],"薬価":row["mean_price"],"brand":0,"generic":0,"otc":1,"in_hospital":0,"oral":row["oral_per"],"同一剤形・規格の後発医薬品がある先発医薬品":0,"Pharmacopoeia":0})
    self_row=row.copy()
    self_row.update({"医薬品名": "self", "メーカー名": "self","総計":row["self_quantity"],"薬価":row["mean_price"],"brand":0,"generic":1,"otc":1,"in_hospital":0,"oral":row["oral_per"],"同一剤形・規格の後発医薬品がある先発医薬品":0,"Pharmacopoeia":0})
    add_data=pd.concat([add_data,pd.DataFrame(nonself_row).T,pd.DataFrame(self_row).T],axis=0)
add_data.reset_index(inplace=True,drop=True)
add_data.shape
data_merged=pd.concat([data_merged,add_data],axis=0)
data_merged.reset_index(inplace=True,drop=True)
print(data_merged.columns[:30],data_merged.shape,add_data.shape)
# add_data.to_csv("tmp.csv",encoding="utf-8")

(200, 134)


Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', 'generic',
       '総計', 'year', 'in_hospital', 'oral', 'id_l4', 'Approved', 'KEGG',
       'Ingredient', 'date', 'release_year', '成分', 'variable', 'id_x',
       'revenue', 'elapsed', 'elapsed_m6', 'elapsed_m5', 'elapsed_m4',
       'elapsed_m3', 'elapsed_m2', 'elapsed_m1', 'elapsed_0', 'elapsed_1'],
      dtype='object') (26198, 134) (400, 134)


In [29]:
# data_merged.loc[data_merged["メーカー名"].isna(),"メーカー名"]=data_merged.loc[data_merged["メーカー名"].isna(),"薬価基準収載医薬品コード"].map(map_dict)
data_rest=data_merged.loc[data_merged["メーカー名"].isna()]
print(len(set(data_rest["薬価基準収載医薬品コード"])-set(firm_data_w_maker["薬価基準収載医薬品コード"])))
len(set(data_rest["薬価基準収載医薬品コード"])),len(set(data_rest["医薬品名"])),len(set(data_merged["薬価基準収載医薬品コード"]))

0


(0, 0, 2671)

In [30]:
len(set(data_rest["医薬品名"]))

0

In [31]:
data_merged.to_csv("../merged/len3_ndb_blp_DN_firm.csv",encoding="utf-8-sig")